## <font color='blue'>Linear regression on California housing data</font>

The California Housing data set was obtained from the 1990 California census. One use of it is to predict housing prices based on features such as house age, location, number of bedrooms, etc.

In the data, the housing has been divided into "blocks", each a geographically compact area containing on average 1400 individuals. There are 20,640 data points, one per block.

Each data point has the following information about the corresponding block:
* median income (multiples of 10K) in that block
* median house age
* average number of rooms in housing in that block
* average number of bedrooms
* population
* average occupancy of houses in block
* latitude
* longitude
* median house value (multiples of 100K)
The regression problem is to predict the house value based on the other 8 features.

### <font color='blue'>1. Loading the data and getting some summary statistics</font>

In addition to `numpy` and `matplotlib` we will be using `pandas`. This gives us a handy way of storing the data in "frames" which include attribute names.

In [1]:
import pandas as pd
import numpy as np

Now let's load in the data and take a quick look at it. The display has one point per row. Notice how nice the formatting is, and how each column is named according to its feature.

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True)
df = housing.frame  # a Pandas data-frame
display(df)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


Now let's look at the correlations between these 9 variables. We can use the `corr()` method in `pandas` for this, and then display the resulting matrix using some nice formatting.

In [3]:
# Compute correlation matrix
corr_matrix = df.corr() 
# Print it nicely
corr_matrix.style \
    .background_gradient(cmap='coolwarm') \
    .format(precision=2)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
MedInc,1.00,-0.12,0.33,-0.06,0.00,0.02,-0.08,-0.02,0.69
HouseAge,-0.12,1.00,-0.15,-0.08,-0.30,0.01,0.01,-0.11,0.11
AveRooms,0.33,-0.15,1.00,0.85,-0.07,-0.00,0.11,-0.03,0.15
AveBedrms,-0.06,-0.08,0.85,1.00,-0.07,-0.01,0.07,0.01,-0.05
Population,0.00,-0.30,-0.07,-0.07,1.00,0.07,-0.11,0.10,-0.02
AveOccup,0.02,0.01,-0.00,-0.01,0.07,1.00,0.00,0.00,-0.02
Latitude,-0.08,0.01,0.11,0.07,-0.11,0.00,1.00,-0.92,-0.14
Longitude,-0.02,-0.11,-0.03,0.01,0.10,0.00,-0.92,1.00,-0.05
MedHouseVal,0.69,0.11,0.15,-0.05,-0.02,-0.02,-0.14,-0.05,1.00


<font color='magenta'>Some questions for you:</font>
* Which (other) feature is most highly correlated with median house value?
* Which pair of features are the most strongly correlated?
* Which pair of features are the most negatively correlated?

### <font color='blue'>2. The regression problem</font>

Next, we'll separate the predictor variables (the first eight columns) from the response variable (the last column). 

We will also split the data into training and test set. There is a nice method for this in `sklearn.model_selection`.

In [4]:
from sklearn.model_selection import train_test_split

# Separate predictor variables (X) from response (y)
X = df.drop(columns=['MedHouseVal'])  # Features
y = df['MedHouseVal']                 # Target

# Split data into training set (X_train, y_train) and test set (X_test, y_test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<font color='magenta'>Some questions for you:</font>
* What are the sizes of the training and test sets?
* Suppose we want to predict `y` (house value) without seeing `x`; what value of `y` would work best for the test set, and what would be the resulting mean squared error on the test set?

In [5]:
print("Mean of y_test:", y_test.mean())
print("Mean Squared Error:", np.mean((y_test - y_test.mean())**2))

Mean of y_test: 2.0550030959302323
Mean Squared Error: 1.3104089782408996


<font color='magenta'> To do: Use `sklearn.linear_model.LinearRegression` to fit a linear function to the training data using least-squares regression. Then display the resulting coefficients of each of the 8 features and give the mean squared error on the test set.

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(X.columns, model.coef_):
    print(f"{feature}: {coef:.4f}")

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on test set: {mse:.4f}")

MedInc: 0.4487
HouseAge: 0.0097
AveRooms: -0.1233
AveBedrms: 0.7831
Population: -0.0000
AveOccup: -0.0035
Latitude: -0.4198
Longitude: -0.4337
Mean Squared Error on test set: 0.5559


<font color='magenta'> To do: Again, we'll fit a linear function (using the training set) and get the mean squared error (on the test set). However, this time we will use just a subset of the features.</font>
* Use just the two features `Latitude` and `Longitude`
* Use just one feature; which is the best choice?

 In both cases, report the resulting mean squared error on the test set.

In [7]:
feature_d = ['Latitude', 'Longitude']
X_train_d = X_train[feature_d]
X_test_d = X_test[feature_d]

model_d = LinearRegression()
model_d.fit(X_train_d, y_train)

y_pred_d = model_d.predict(X_test_d)
mse = mean_squared_error(y_test, y_pred_d)
print(f"Mean Squared Error on test set: {mse:.4f}")

Mean Squared Error on test set: 0.9788


In [8]:
mse_score = {}

for feature in X.columns:
    X_train_f = X_train[[feature]]
    X_test_f = X_test[[feature]]
    
    model_f = LinearRegression()
    model_f.fit(X_train_f, y_train)
    
    y_pred_f = model_f.predict(X_test_f)
    mse = mean_squared_error(y_test, y_pred_f)
    mse_score[feature] = mse

    print(f"Feature: {feature}, Mean Squared Error: {mse:.4f}")

best_feature = min(mse_score, key=mse_score.get)
best_mse = mse_score[best_feature]
print(f"Best single feature: {best_feature} Mean Squared Error: {best_mse:.4f}")

Feature: MedInc, Mean Squared Error: 0.7091
Feature: HouseAge, Mean Squared Error: 1.2940
Feature: AveRooms, Mean Squared Error: 1.2923
Feature: AveBedrms, Mean Squared Error: 1.3109
Feature: Population, Mean Squared Error: 1.3103
Feature: AveOccup, Mean Squared Error: 1.3096
Feature: Latitude, Mean Squared Error: 1.2817
Feature: Longitude, Mean Squared Error: 1.3081
Best single feature: MedInc Mean Squared Error: 0.7091


In [9]:
from sklearn.linear_model import LassoCV

# --- 1. Load the data ---
# Make sure 'mystery.dat' is in the same directory as your script
try:
    data = np.loadtxt('mystery.dat', delimiter=',')
    
    # --- 2. Separate features (X) and target (y) ---
    # X is all columns except the last one
    X = data[:, :-1]
    # y is only the last column
    y = data[:, -1]

    # --- 3. Create and fit the LassoCV model ---
    # LassoCV uses cross-validation to find the best alpha (regularization strength)
    # n_alphas=200 increases the chance of finding the sparse solution
    model = LassoCV(n_alphas=200, cv=10, random_state=42)
    model.fit(X, y)

    # --- 4. Find the non-zero coefficients ---
    coefficients = model.coef_
    
    # np.where returns the indices where the condition (coef != 0) is true
    nonzero_indices = np.where(coefficients != 0)[0]
    
    # Add 1 to the 0-based indices to get 1-based coordinate numbers
    feature_numbers = nonzero_indices + 1

    print("--- Lasso Regression Results ---")
    print(f"Optimal alpha (regularization strength) found: {model.alpha_:.5f}")
    print(f"Total features selected: {len(feature_numbers)}")
    print("\nIdentified feature coordinates (1-100):")
    print(sorted(feature_numbers))

except FileNotFoundError:
    print("Error: 'mystery.dat' not found.")
    print("Please download the file and place it in the same directory.")
except Exception as e:
    print(f"An error occurred: {e}")

/opt/anaconda3/envs/cse151a/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:1622: FutureWarning: 'n_alphas' was deprecated in 1.7 and will be removed in 1.9. 'alphas' now accepts an integer value which removes the need to pass 'n_alphas'. The default value of 'alphas' will change from None to 100 in 1.9. Pass an explicit value to 'alphas' and leave 'n_alphas' to its default value to silence this warning.
  warnings.warn(


--- Lasso Regression Results ---
Optimal alpha (regularization strength) found: 0.13161
Total features selected: 21

Identified feature coordinates (1-100):
[np.int64(1), np.int64(2), np.int64(3), np.int64(5), np.int64(7), np.int64(11), np.int64(13), np.int64(17), np.int64(19), np.int64(23), np.int64(26), np.int64(27), np.int64(35), np.int64(36), np.int64(43), np.int64(46), np.int64(55), np.int64(60), np.int64(82), np.int64(86), np.int64(88)]


In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import warnings

# Suppress warnings for cleaner output
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# --- 1. Load and Prepare Data ---
data = pd.read_csv('heart.csv', header=0)

# Now we must find the label column by its name, not just 'the last one'.
# We can get its name to drop it from X.
target_col_name = data.columns[-1]

X = data.drop(columns=[target_col_name])
y = data[target_col_name]

# Get feature names from the loaded header
feature_names = X.columns

# --- (a) Partition data and fit the model ---
# Randomly partition into 200 training and 103 test points
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                train_size=200, 
                                                test_size=103, 
                                                random_state=42)

# We MUST scale data for logistic regression, which uses regularization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

print("--- (a) Model Coefficients ---")
# Display coefficients
coef_df = pd.DataFrame(model.coef_[0], index=feature_names, columns=['Coefficient'])
print(coef_df)

--- (a) Model Coefficients ---
          Coefficient
age          0.169770
sex         -0.628130
cp           0.932279
trestbps    -0.148110
chol        -0.069233
fbs          0.095300
restecg      0.309317
thalach      0.376398
exang       -0.507671
oldpeak     -0.489070
slope        0.534152
ca          -1.311742
thal        -0.788929


In [11]:
# --- (b) Three most influential features ---
print("\n--- (b) Most Influential Features ---")
# We find the features with the largest absolute coefficient value
# This is a valid method *because* we scaled the data
coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
influential_features = coef_df.sort_values(by='Abs_Coefficient', ascending=False)

print("The three most influential features (by absolute coefficient) are:")
print(influential_features.head(3))


--- (b) Most Influential Features ---
The three most influential features (by absolute coefficient) are:
      Coefficient  Abs_Coefficient
ca      -1.311742         1.311742
cp       0.932279         0.932279
thal    -0.788929         0.788929


In [12]:
# --- (c) Test error ---
print("\n--- (c) Test Error ---")
y_pred = model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred)
test_error = 1 - test_accuracy

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Error (1 - Accuracy): {test_error:.4f}")


--- (c) Test Error ---
Test Accuracy: 0.8155
Test Error (1 - Accuracy): 0.1845


In [13]:
# --- (d) 5-fold cross-validation ---
print("\n--- (d) 5-Fold Cross-Validation Error ---")
# A Pipeline bundles the scaler and model to handle this automatically.
pipeline = Pipeline([
('scaler', StandardScaler()),
('model', LogisticRegression(random_state=42))
])

# Run 5-fold CV on the *training set*
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

print(f"Individual CV (accuracy) scores: {cv_scores}")

# Estimate the error from the mean of the CV scores
estimated_accuracy = np.mean(cv_scores)
estimated_error = 1 - estimated_accuracy

print(f"\nEstimated Accuracy (mean of CV scores): {estimated_accuracy:.4f}")
print(f"Estimated Error (from CV): {estimated_error:.4f}")

# Comparison
print("\n--- Comparison ---")
print(f"Test Error (from single split): {test_error:.4f}")
print(f"Estimated Error (from 5-fold CV): {estimated_error:.4f}")


--- (d) 5-Fold Cross-Validation Error ---
Individual CV (accuracy) scores: [0.85  0.85  0.85  0.875 0.8  ]

Estimated Accuracy (mean of CV scores): 0.8450
Estimated Error (from CV): 0.1550

--- Comparison ---
Test Error (from single split): 0.1845
Estimated Error (from 5-fold CV): 0.1550
